In [1]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings.cohere import CohereEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores.elastic_vector_search import ElasticVectorSearch
from langchain.vectorstores.faiss import FAISS
from langchain.docstore.document import Document

In [2]:
with open('../state_of_the_union.txt') as f:
    state_of_the_union = f.read()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_text(state_of_the_union)

embeddings = OpenAIEmbeddings()

In [3]:
docsearch = FAISS.from_texts(texts, embeddings)

In [4]:
query = "What did the president say about Justice Breyer"
docs = docsearch.similarity_search(query)

In [5]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

In [6]:
chain = load_qa_chain(OpenAI(temperature=0), chain_type="stuff")

In [7]:
docs = [Document(page_content=t) for t in texts[:3]]

In [8]:
query = "What did the president say about Justice Breyer"
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

{'output_text': ' The president did not mention Justice Breyer.'}

In [9]:
chain = load_qa_chain(OpenAI(temperature=0), chain_type="map_reduce")

In [10]:
query = "What did the president say about Justice Breyer"
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

{'output_text': ' The president did not mention Justice Breyer.'}

In [8]:
chain = load_qa_chain(OpenAI(temperature=0), chain_type="refine")

In [10]:
query = "What did the president say about Justice Breyer"
chain({"input_documents": docs, "query_str": query}, return_only_outputs=True)

{'output_text': '\n\nThe president did not mention Justice Breyer in the given page content.'}